# Parking Lots Setup

In [1]:
import os
import math
import csv
import json
# import pandas as pd
# import numpy as np
from datetime import datetime

In [2]:
'''
A spot can be - single
Can have subclasses - Single, Twin, Station (5)
'''
class ParkingSpot(object):
    
    '''
    EVENT - car leaves
    RATE - minute basis
    '''
    '''
    ParkingSpot(meterhead, rates)
    meterhead a string specifying the meterhead
    rates a list of 6 numbers being the cost in dollars (MF 9-6, MF 6-10, SA 9-6, SA 6-10, SU 9-6, SU 6-10)
    limits a list of 6 numbers (similarly to rates) indicating how many hour we can stay parked in this spot
    payphone a int payphone number
    geo being a list of 2 numbers: lat and long
    '''
    def __init__(self, meterhead, rates, limits, payphone, geo, usage_multiplier):
        if meterhead[0] == "S":
            # Single
            self.capacity = 1
        elif meterhead[0] == "T":
            # Twin
            self.capacity = 2
        else:
            # Pay Station
            self.capacity = 10
        i = 0
        hour = datetime.now().hour
        if hour >= 9 and hour < 6:
            i = 1
        weekday = datetime.today().weekday()
        if weekday == 5:
            # Saturday
            i += 2
        elif weekday == 6:
            # Sunday
            i += 4
        self.meter_rate = rates[i]/60 #Per minute
        self.meter_limits = limits[i]*60 #How long (in minutes)
        if hour < 9 and hour >= 22:
            # Afterhours
            self.meter_rate = 0
            if hour >= 22:
                self.meter_limits = (24-hour+9)*60
            else:
                self.meter_limits = (9-hour)*60
        self.lam_depart = usage_multiplier*4/60 # cars leaving per hour per lot (see capacity)
        self.payphone = payphone
        self.geo = geo
    
    '''
    Distance to a given geometric location in meters
    '''
    def distance(self, geo):
        # https://en.wikipedia.org/wiki/Haversine_formula
        my_lat,my_lon = map(math.radians, self.geo)
        dest_lat,dest_lon = map(math.radians, geo)
        fn = (  
             math.sin((dest_lat - my_lat) / 2) ** 2 
           + math.cos(my_lat) 
           * math.cos(dest_lat) 
           * math.sin((dest_lon - my_lon ) / 2) ** 2
        )
        return 6373000.0 * (2 * math.atan2(math.sqrt(fn), math.sqrt(1 - fn)))
    
    '''
    Helper function to display this entity
    '''
    def __str__(self):
        return f"Parking Spot #{self.payphone} at {self.geo} with capacity for {self.capacity} and limit of {self.meter_limits}min"
    def __repr__(self):
        return f"Parking Spot #{self.payphone} capacity:{self.capacity} limit:{self.meter_limits}"
    
    '''
    Return available spots
    '''
    def available(self):
        return (self.capacity + int(self.payphone[:-1])) % self.capacity # TODO

List parking

In [3]:
def readPrice(data):
    if len(data) == 0:
        return 0.0
    return float(data[1:])
assert readPrice("$2.50")==2.5

def readTime(data):
    if len(data) == 0 or data == "No Time Limit" or data == "other":
        return 24.0*60.0
    return 60.0*float(data[:-3])
assert readTime("2 Hr") == 60.0*2

def readGeo(data):
    try:
        return json.loads(data)["coordinates"]
    except:
        return [0,0]
assert readGeo('{"coordinates": [-123.14344688271457, 49.26438353678866], "type": "Point"}')==[-123.14344688271457, 49.26438353678866]

parking_spots = []
with open('data/parking-meters.csv', mode ='r') as file: 
    csvFile = csv.reader(file, delimiter=";")
    first=True
    for lines in csvFile:
        if first:
            first = False
            continue
        parking_spots.append(ParkingSpot(
            lines[0],
            [readPrice(lines[1]), readPrice(lines[2]), readPrice(lines[3]), readPrice(lines[4]), readPrice(lines[5]), readPrice(lines[6])],
            [readTime(lines[9]), readTime(lines[10]), readTime(lines[11]), readTime(lines[12]), readTime(lines[13]), readTime(lines[14])],
            lines[17],
            readGeo(lines[18]),
            1
        ))

Filter and sort parking list

In [4]:
'''
Filters a list based on the user requested parking time and expected availability of the parking lot
'''
def filterSpots(parking_spots, user_parking_time):
    def availability_filter(ps):
        if ps.available() >= 1:
            return True
        return False
    def time_filter(ps):
        if ps.meter_limits >= user_parking_time:
            return True
        return False
    ls = list(filter(availability_filter, parking_spots))
    print(f"{len(ls)} parking locations with available spots")
    ls = list(filter(time_filter, ls))
    print(f"{len(ls)} parking location available for long enough")
    return ls

'''
Returns an ordered list based on distance to a target
'''
def OrderSpots(parking_spots, target):
    def distanceSorter(ps):
        return ps.distance(target)
    parking_spots.sort(key=distanceSorter)
    return parking_spots

print(f"{len(parking_spots)} parking locations")
parking_spots = OrderSpots(filterSpots(parking_spots, 2.0 * 60.0), [49.2664327,-123.2557465])

print(f"Top Location ({int(parking_spots[0].distance([49,-123]))}mt from user):")
print(parking_spots[0])

print("Top 10:")
print(parking_spots[0:10])

7954 parking locations
3757 parking locations with available spots
3757 parking location available for long enough
Top Location (11788474mt from user):
Parking Spot #51636 at [-123.21180416536625, 49.263949007848616] with capacity for 2 and limit of 7200.0min
Top 10:
[Parking Spot #51636 capacity:2 limit:7200.0, Parking Spot #52257 capacity:2 limit:7200.0, Parking Spot #51751 capacity:2 limit:7200.0, Parking Spot #52291 capacity:2 limit:7200.0, Parking Spot #50714 capacity:2 limit:7200.0, Parking Spot #53414 capacity:2 limit:7200.0, Parking Spot #50854 capacity:2 limit:7200.0, Parking Spot #51217 capacity:2 limit:7200.0, Parking Spot #50856 capacity:2 limit:7200.0, Parking Spot #53036 capacity:2 limit:7200.0]
